In [1]:
import pandas as pd
import numpy as np
import datetime

In [62]:
path = 'resources/pacemed/'

df1 = pd.read_csv(path+'age.csv', index_col=None, sep=';')
df2 = pd.read_csv(path+'admission.csv', index_col=None, sep=';')
df3 = pd.read_csv(path+'signal.csv', index_col=None, sep=';')

df1.drop(['Unnamed: 0'], axis=1, inplace=True)
df2.drop(['Unnamed: 0'], axis=1, inplace=True)
df3.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [51]:
df2['date_admission'] = df2['date_admission'].astype('datetime64')
df2['date_discharge']= df2['date_discharge'].astype('datetime64')
df_admission = df2.groupby('pat_id')['date_admission'].agg('count').to_frame('Count').reset_index()
df_admission = pd.merge(df2, df_admission, on='pat_id')
df_admission = pd.merge(df_admission, df1, on='pat_id')
df_admission['days_ICU'] = (df_admission['date_discharge']-df_admission['date_admission']).astype('timedelta64[D]')
#df_admission.to_csv(path+'test_admission.csv', sep=',', index=None)
primary_key = df_admission['pat_id']+df_admission['date_admission'].astype('int')
df_admission.insert(loc=0, column='primary_key', value=primary_key)
df_admission['bad_discharge'] = [0]*df_admission.shape[0]

In [64]:
for pat in np.unique(df_admission['pat_id']):
    source_rows = df_admission.index[df_admission['pat_id']==pat].tolist()
    if(len(source_rows)>1):
        for i in range(0, len(source_rows)):
            if (i != len(source_rows)-1):
                if((df_admission['date_admission'].iloc[source_rows[i+1]] - df_admission['date_discharge'].iloc[source_rows[i]]).days < 60):
                    df_admission['bad_discharge'].iloc[source_rows[i]] = 1
# df_temp = df_admission[df_admission['bad_discharge']==1]
# print(df_temp)                   
df3['day'] = df3['day'].astype('datetime64')
df3.insert(loc=0, column='primary_key', value=[0]*df3.shape[0])

In [ ]:

for pat in np.unique(df_admission['pat_id']):
    source_rows = df_admission.index[df_admission['pat_id']==pat].tolist()
    for i in range(len(source_rows)):
        tgt_rows = df3.index[(df3['pat_id']==pat) & \
                   (df_admission['date_admission'].iloc[source_rows[i]] <= df3['day']) & \
                   (df3['day'] <= df_admission['date_discharge'].iloc[source_rows[i]])].tolist()
        df3['primary_key'].iloc[tgt_rows] = [df_admission['primary_key'].iloc[source_rows[i]]]*len(tgt_rows) 

In [74]:
df_admission.to_csv(path+'test_admission.csv', sep=',',index=None)
df3.to_csv(path+'test_signal.csv', sep=',', index=None)

In [94]:
df_signal = df3.groupby(['primary_key', 'parameter']).mean()['value']
df_signal = df_signal.unstack().reset_index()
df_signal.columns.name = None
df_semifinal = df_signal.merge(df_admission[['primary_key', 'age', 'days_ICU', 'bad_discharge']], on='primary_key', how='right')
df_semifinal.to_csv(path+"collated.csv", sep=',', index=None)
#df_signal.head()
df_semifinal.tail()

,primary_key,blood_pressure,respiration_rate,temperature,age,days_ICU,bad_discharge
1495,1.473466e+18,NaN,NaN,NaN,73.0,1.0,0
1496,1.470874e+18,NaN,NaN,NaN,42.0,1.0,0
1497,1.471738e+18,NaN,NaN,NaN,61.0,6.0,0
1498,1.470096e+18,NaN,NaN,NaN,34.0,1.0,0
1499,1.471824e+18,NaN,NaN,NaN,55.0,2.0,0
